In [27]:
a = 1+1

2

In [28]:
using JudiLing
using CSV # read CSV file
using DataFrames # load CSV file into DataFrames

In [29]:
estonian_train = DataFrame(CSV.File(joinpath(@__DIR__, "estonian_train.csv")))
estonian_val = DataFrame(CSV.File(joinpath(@__DIR__, "estonian_val.csv")));

In [4]:
estonian_train

,Word,Lexeme,Case,Number,Translation,pos
,String,String,String,String,String,String
1,ümbrikul,ümbrik,ad,sg,envelope,6000
2,rüüdele,rüü,all,pl,robe,6293
3,jõesse,jõgi,ill,sg,river,4988
4,koer,koer,nom,sg,dog,5433
5,seenega,seen,kom,sg,mushroom,3290
6,olulisest,oluline,el,sg,important,2946
7,sohvadena,sohva,es,pl,couch,3694
8,vari,vari,nom,sg,shadow,5825
9,mitmena,mitu,es,sg,many,1328


In [30]:
estonian_val

,Word,Lexeme,Case,Number,Translation,pos
,String,String,String,String,String,Int64
1,asemetes,ase,in,pl,stead,859
2,asemeteta,ase,abes,pl,stead,867
3,sugudena,sugu,es,pl,gender,4338
4,tuvi,tuvi,nom,sg,pidgeon,4033
5,ümbrikutele,ümbrik,all,pl,envelope,6013
6,varteni,vars,term,pl,shaft,3469
7,varrelt,vars,abl,sg,shaft,3453
8,ronimiseni,ronimine,term,sg,climbing,3203
9,koerteni,koer,term,pl,dog,5457


In [35]:
@time cue_obj_train, cue_obj_val = JudiLing.make_cue_matrix(
    estonian_train,
    estonian_val,
    grams=3,
    target_col=:Word,
    tokenized=false
    );

  0.053405 seconds (928.31 k allocations: 55.206 MiB)


In [32]:
JudiLing.display_SC(estonian_train[:,:Word], cue_obj_train)

,Data,#üm,ümb,mbr,bri,rik,iku
,String,Int64,Int64,Int64,Int64,Int64,Int64
1,ümbrikul,1,1,1,1,1,1
2,rüüdele,0,0,0,0,0,0
3,jõesse,0,0,0,0,0,0
4,koer,0,0,0,0,0,0
5,seenega,0,0,0,0,0,0
6,olulisest,0,0,0,0,0,0


In [36]:
n_features = size(cue_obj_train.C, 2)
@time S_train, S_val = JudiLing.make_S_matrix(
    estonian_train,
    estonian_val,
    ["Lexeme"],
    ["Case","Number"],
    ncol=n_features,
    add_noise=true
    );

  0.437050 seconds (236.61 k allocations: 581.938 MiB, 15.41% gc time)


In [34]:
JudiLing.display_SC(estonian_train[:,:Word], S_train)

,Data,S1,S2,S3,S4,S5,S6
,String,Float64,Float64,Float64,Float64,Float64,Float64
1,ümbrikul,10.4098,-12.4237,4.70806,4.87667,7.05879,-5.67645
2,rüüdele,-0.727067,-8.46497,-0.590723,-0.931539,12.83,1.44924
3,jõesse,-4.4627,6.07277,-9.73858,11.0912,6.63827,-0.615547
4,koer,-12.4006,-11.3757,-0.433701,3.60604,7.77946,-6.87679
5,seenega,2.50122,-6.69969,14.548,-1.01514,1.2776,-5.03294
6,olulisest,-5.6412,-5.4344,3.76818,15.0093,13.8653,-14.3231


In [40]:
@time F_train = JudiLing.make_transform_matrix(cue_obj_train.C, S_train)

  0.333925 seconds (81 allocations: 138.420 MiB)


1418×1418 Array{Float64,2}:
  0.075295   -1.53387   -0.376434    …   1.10242     5.12444     1.50063
  0.075295   -1.53387   -0.376434        1.10242     5.12444     1.50063
  5.09693    -1.12226    0.207173        0.81646     2.20127    -1.67911
  2.51339    -1.48637   -0.00742256     -0.0747007   1.01625    -0.512721
  1.20928    -0.420672   1.47649         0.771718    0.611372   -0.276218
  1.79374     0.366472  -0.376242    …  -0.470863   -1.1305     -1.09491
  0.529348   -0.739404   1.24749         0.742485   -0.452001    0.962457
  0.74393    -6.32098    2.61241         2.21226    -1.90816    -3.69313
  0.408837   -3.61282    0.706432       -1.06063    -0.44103     2.25795
  0.408837   -3.61282    0.706432       -1.06063    -0.44103     2.25795
 -3.00055     0.176644   1.98796     …   1.99747     4.29569     1.18051
  1.35035     1.12934   -1.36923        -1.22444     0.0128092  -1.84215
  0.547951   -0.349804  -1.19915        -0.343105   -2.02398    -3.19108
  ⋮                 

In [42]:
@time Shat_train = cue_obj_train.C * F_train
@time Shat_val = cue_obj_val.C * F_train;

  0.097001 seconds (3 allocations: 55.748 MiB)
  0.069083 seconds (3 allocations: 13.934 MiB, 41.25% gc time)


In [43]:
@show JudiLing.eval_SC(Shat_train, S_train, estonian_train, :Word)
@show JudiLing.eval_SC(Shat_val, S_val, estonian_val, :Word);

JudiLing.eval_SC(Shat_train, S_train, estonian_train, :Word) = 0.9920434698234039
JudiLing.eval_SC(Shat_val, S_val, estonian_val, :Word) = 0.9836956521739131


In [44]:
@time G_train = JudiLing.make_transform_matrix(S_train, cue_obj_train.C);

  0.319681 seconds (20 allocations: 122.737 MiB)


In [45]:
Chat_train = S_train * G_train
Chat_val = S_val * G_train;

In [15]:
A = cue_obj_train.A

max_t = JudiLing.cal_max_timestep(estonian_train, estonian_val, :Word);

In [46]:
@time res_learn_train = JudiLing.learn_paths(
    estonian_train,
    estonian_train,
    cue_obj_train.C,
    S_train,
    F_train,
    Chat_train,
    A,
    cue_obj_train.i2f,
    cue_obj_train.f2i,
    max_t=max_t,
    max_can=10,
    grams=3,
    threshold=0.05,
    tokenized=false,
    keep_sep=false,
    target_col=:Word,
    verbose=true);

Making fac C
Timestep 1
Calculating Yt...
Calculating Mt...
Auto mode: Sparsity: 0.07334273624823695
Returning a sparse matrix format
Calculating Ythat...
Sparsity: 0.07334273624823695
Finding paths...


Progress: 100%|█████████████████████████████████████████| Time: 0:00:00


Timestep 2
average 1.0890743256355522 of paths currently
Calculating Yt...
Calculating Mt...
Auto mode: Sparsity: 0.1755994358251058
Returning a sparse matrix format
Calculating Ythat...
Sparsity: 0.1755994358251058
Finding paths...


Progress: 100%|█████████████████████████████████████████| Time: 0:00:00


Timestep 3
average 1.1542790607413158 of paths currently
Calculating Yt...
Calculating Mt...
Auto mode: Sparsity: 0.2834978843441467
Returning a dense matrix format
Calculating Ythat...
Sparsity: 0.2834978843441467
Finding paths...


...

Timestep 18
average 0.0 of paths currently
Calculating Yt...
Calculating Mt...
Auto mode: Sparsity: 4.97333298851558e-7
Returning a sparse matrix format
Calculating Ythat...
Sparsity: 0.0
Finding paths...


Progress: 100%|█████████████████████████████████████████| Time: 0:00:00


Evaluating paths...
average 4.157772171550553 of paths to evaluate


Progress: 100%|█████████████████████████████████████████| Time: 0:00:04


 35.753492 seconds (13.42 M allocations: 10.813 GiB, 4.48% gc time)


In [47]:
@time res_learn_val = JudiLing.learn_paths(
    estonian_train,
    estonian_val,
    cue_obj_train.C,
    S_val,
    F_train,
    Chat_val,
    A,
    cue_obj_train.i2f,
    cue_obj_train.f2i,
    max_t=max_t,
    max_can=10,
    grams=3,
    threshold=0.05,
    is_tolerant=true,
    tolerance=-0.1,
    max_tolerance=3,
    tokenized=false,
    keep_sep=false,
    target_col=:Word,
    verbose=true);

Making fac C
Timestep 1
Calculating Yt...
Calculating Mt...
Auto mode: Sparsity: 0.07334273624823695
Returning a sparse matrix format
Calculating Ythat...
Sparsity: 0.07334273624823695
Finding paths...


Progress: 100%|█████████████████████████████████████████| Time: 0:00:01


Timestep 2
average 103.99767080745342 of paths currently
Calculating Yt...
Calculating Mt...
Auto mode: Sparsity: 0.1755994358251058
Returning a sparse matrix format
Calculating Ythat...
Sparsity: 0.1755994358251058
Finding paths...


Progress: 100%|█████████████████████████████████████████| Time: 0:00:03


Timestep 3
average 246.97903726708074 of paths currently
Calculating Yt...
Calculating Mt...
Auto mode: Sparsity: 0.2834978843441467
Returning a dense matrix format
Calculating Ythat...
Sparsity: 0.2834978843441467
Finding paths...


...

Timestep 14
average 9.362577639751553 of paths currently
Calculating Yt...
Calculating Mt...
Auto mode: Sparsity: 0.012693935119887164
Returning a sparse matrix format
Calculating Ythat...
Sparsity: 0.012693935119887164
Finding paths...


Progress: 100%|█████████████████████████████████████████| Time: 0:00:00


Timestep 15
average 5.71972049689441 of paths currently
Calculating Yt...
Calculating Mt...
Auto mode: Sparsity: 0.007757404795486601
Returning a sparse matrix format
Calculating Ythat...
Sparsity: 0.007757404795486601
Finding paths...
Timestep 16
average 1.9821428571428572 of paths currently
Calculating Yt...
Calculating Mt...
Auto mode: Sparsity: 0.005641748942172073
Returning a sparse matrix format
Calculating Ythat...
Sparsity: 0.005641748942172073
Finding paths...
Timestep 17
average 0.5551242236024845 of paths currently
Calculating Yt...
Calculating Mt...
Auto mode: Sparsity: 4.97333298851558e-7
Returning a sparse matrix format
Calculating Ythat...
Sparsity: 0.0
Finding paths...
Timestep 18
average 0.12111801242236025 of paths currently
Calculating Yt...
Calculating Mt...
Auto mode: Sparsity: 4.97333298851558e-7
Returning a sparse matrix format
Calculating Ythat...
Sparsity: 0.0
Finding paths...
Evaluating paths...
average 209.85170807453417 of paths to evaluate


Progress: 100%|█████████████████████████████████████████| Time: 0:00:36


 66.813921 seconds (66.64 M allocations: 34.088 GiB, 6.60% gc time)


In [48]:
acc_learn_train = JudiLing.eval_acc(
  res_learn_train,
  cue_obj_train.gold_ind,
  verbose=false
)

acc_learn_val = JudiLing.eval_acc(
  res_learn_val,
  cue_obj_val.gold_ind,
  verbose=false
)

@show acc_learn_train
@show acc_learn_val
;

acc_learn_train = 0.9881622355909179
acc_learn_val = 0.8027950310559007


In [51]:
A = JudiLing.make_combined_adjacency_matrix(
    estonian_train[:,[:Word]],
    estonian_val[:,[:Word]],
    grams=3,
    target_col=:Word,
    tokenized=false,
    keep_sep=false
    );

In [52]:
@time res_learn_val = JudiLing.learn_paths(
    estonian_train,
    estonian_val,
    cue_obj_train.C,
    S_val,
    F_train,
    Chat_val,
    A,
    cue_obj_train.i2f,
    cue_obj_train.f2i,
    max_t=max_t,
    max_can=10,
    grams=3,
    threshold=0.05,
    is_tolerant=true,
    tolerance=-0.1,
    max_tolerance=3,
    tokenized=false,
    keep_sep=false,
    target_col=:Word,
    verbose=true)

acc_learn_val = JudiLing.eval_acc(
  res_learn_val,
  cue_obj_val.gold_ind,
  verbose=false
)

@show acc_learn_val
;

Making fac C
Timestep 1
Calculating Yt...
Calculating Mt...
Auto mode: Sparsity: 0.07334273624823695
Returning a sparse matrix format
Calculating Ythat...
Sparsity: 0.07334273624823695
Finding paths...


Progress: 100%|█████████████████████████████████████████| Time: 0:00:01


Timestep 2
average 103.99767080745342 of paths currently
Calculating Yt...
Calculating Mt...
Auto mode: Sparsity: 0.1755994358251058
Returning a sparse matrix format
Calculating Ythat...
Sparsity: 0.1755994358251058
Finding paths...


Progress: 100%|█████████████████████████████████████████| Time: 0:00:03


Timestep 3
average 246.97903726708074 of paths currently
Calculating Yt...
Calculating Mt...
Auto mode: Sparsity: 0.2834978843441467
Returning a dense matrix format
Calculating Ythat...
Sparsity: 0.2834978843441467
Finding paths...


...

Timestep 14
average 9.796583850931677 of paths currently
Calculating Yt...
Calculating Mt...
Auto mode: Sparsity: 0.012693935119887164
Returning a sparse matrix format
Calculating Ythat...
Sparsity: 0.012693935119887164
Finding paths...


Progress: 100%|█████████████████████████████████████████| Time: 0:00:00


Timestep 15
average 6.008540372670807 of paths currently
Calculating Yt...
Calculating Mt...
Auto mode: Sparsity: 0.007757404795486601
Returning a sparse matrix format
Calculating Ythat...
Sparsity: 0.007757404795486601
Finding paths...
Timestep 16
average 2.073757763975155 of paths currently
Calculating Yt...
Calculating Mt...
Auto mode: Sparsity: 0.005641748942172073
Returning a sparse matrix format
Calculating Ythat...
Sparsity: 0.005641748942172073
Finding paths...
Timestep 17
average 0.5768633540372671 of paths currently
Calculating Yt...
Calculating Mt...
Auto mode: Sparsity: 4.97333298851558e-7
Returning a sparse matrix format
Calculating Ythat...
Sparsity: 0.0
Finding paths...
Timestep 18
average 0.12732919254658384 of paths currently
Calculating Yt...
Calculating Mt...
Auto mode: Sparsity: 4.97333298851558e-7
Returning a sparse matrix format
Calculating Ythat...
Sparsity: 0.0
Finding paths...
Evaluating paths...
average 218.13509316770185 of paths to evaluate


Progress: 100%|█████████████████████████████████████████| Time: 0:00:38


 71.975897 seconds (67.68 M allocations: 35.242 GiB, 6.32% gc time)
acc_learn_val = 0.9013975155279503


In [53]:
@time res_build_train = JudiLing.build_paths(
    estonian_train,
    cue_obj_train.C,
    S_train,
    F_train,
    Chat_train,
    A,
    cue_obj_train.i2f,
    cue_obj_train.gold_ind,
    max_t=max_t,
    n_neighbors=3,
    verbose=true
    )

@time res_build_val = JudiLing.build_paths(
    estonian_val,
    cue_obj_train.C,
    S_val,
    F_train,
    Chat_val,
    A,
    cue_obj_train.i2f,
    cue_obj_train.gold_ind,
    max_t=max_t,
    n_neighbors=20,
    verbose=true
    );

Finding all top features..
finding all n_neighbors features...


Progress: 100%|█████████████████████████████████████████| Time: 0:00:02


Finding paths...


Progress: 100%|█████████████████████████████████████████| Time: 0:00:00


Evaluating paths...
average 4.272850766543761 of paths to evaluate


...

Finding paths...


Progress: 100%|█████████████████████████████████████████| Time: 0:00:00


Evaluating paths...
average 119.5194099378882 of paths to evaluate


Progress: 100%|█████████████████████████████████████████| Time: 0:00:25


 26.838898 seconds (9.10 M allocations: 27.607 GiB, 9.21% gc time)


In [54]:
acc_build_train = JudiLing.eval_acc(
  res_build_train,
  cue_obj_train.gold_ind,
  verbose=false
)

acc_build_val = JudiLing.eval_acc(
  res_build_val,
  cue_obj_val.gold_ind,
  verbose=false
)

@show acc_build_train
@show acc_build_val
;

acc_build_train = 0.9749660392004658
acc_build_val = 0.406055900621118


In [55]:
@time JudiLing.test_combo(
    :pre_split,
    data_dir_path=joinpath(@__DIR__),
    data_prefix="estonian",
    n_features_columns=["Lexeme","Case","Number"],
    n_grams_target_col=:Word,
    n_grams_tokenized=false,
    grams=3,
    n_features_base = ["Lexeme"],
    n_features_inflections = ["Lexeme","Case","Number"],
    if_combined = false,
    A_mode = :train_only,
    threshold_train = 0.05,
    is_tolerant_train = false,
    threshold_val = 0.1,
    is_tolerant_val = true,
    tolerance_val = -0.1,
    max_tolerance_val = 3,
    verbose = true)

making adjacency matrix...
Making S matrix...
Auto mode: Sparsity: 1.0
Returning a dense matrix format
Auto mode: Sparsity: 1.0
Returning a dense matrix format
Making fac C
Timestep 1
Calculating Yt...
Calculating Mt...
Auto mode: Sparsity: 0.07334273624823695
Returning a dense matrix format
Calculating Ythat...
Sparsity: 0.07334273624823695
Finding paths...


Progress: 100%|█████████████████████████████████████████| Time: 0:00:00


Timestep 2
average 1.0397826508829808 of paths currently
Calculating Yt...
Calculating Mt...
Auto mode: Sparsity: 0.1755994358251058
Returning a dense matrix format
Calculating Ythat...
Sparsity: 0.1755994358251058
Finding paths...


Progress: 100%|█████████████████████████████████████████| Time: 0:00:00


Timestep 3
average 1.1538909373180672 of paths currently
Calculating Yt...
Calculating Mt...
Auto mode: Sparsity: 0.2834978843441467
Returning a dense matrix format
Calculating Ythat...
Sparsity: 0.2834978843441467
Finding paths...


...

Finding paths...


Progress: 100%|█████████████████████████████████████████| Time: 0:00:00


Evaluating paths...
average 130.3416149068323 of paths to evaluate


Progress: 100%|█████████████████████████████████████████| Time: 0:00:27


127.739193 seconds (111.36 M allocations: 95.446 GiB, 8.61% gc time)


In [56]:
f = open("out/acc.out")
for i in 1:10
    @show readline(f)
end

readline(f) = "Acc for Chat train: 0.9126722297690666"
readline(f) = "Acc for Shat train: 0.9749660392004658"
readline(f) = "Acc for Shat train homophones: 0.9912672229769066"
readline(f) = "Acc for Chat val: 0.8260869565217391"
readline(f) = "Acc for Shat val: 0.9798136645962733"
readline(f) = "Acc for Shat val homophones: 0.9860248447204969"
readline(f) = "Acc for learn_path train: 0.9827285076654376"
readline(f) = "Acc for learn_path val: 0.7919254658385093"
readline(f) = "Acc for build_path train: 0.9780710265864545"
readline(f) = "Acc for build_path val: 0.38742236024844723"
